# CounterFactual Experiment

## Import Libraries

In [ ]:
from tqdm import tqdm
from IPython.display import HTML, display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import importlib
import itertools
import warnings
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.utils import Bunch
from typing import cast
from CounterFactualModel import CounterFactualModel
from ConstraintParser import ConstraintParser
import CounterFactualVisualizer as CounterFactualVisualizer
importlib.reload(CounterFactualVisualizer)
from CounterFactualVisualizer import (plot_pca_with_counterfactual, plot_sample_and_counterfactual_heatmap, 
                                     plot_pca_loadings, plot_constraints, 
                                     plot_sample_and_counterfactual_comparison, plot_pairwise_with_counterfactual_df,
                                     plot_pca_with_counterfactuals, plot_explainer_summary)
from CounterFactualExplainer import CounterFactualExplainer

warnings.filterwarnings("ignore")

In [ ]:
# Configure matplotlib for inline display and create helper function
%matplotlib inline

def display_figure(fig):
    """Helper function to properly display matplotlib figures loaded from pickle"""
    if fig is not None:
        # For matplotlib figures, we need to explicitly show them
        if hasattr(fig, 'canvas'):
            from IPython.display import display
            display(fig)
            # Force render
            fig.canvas.draw()
        else:
            display(fig)

In [ ]:
# Storage helpers using the per-sample layout (delegates to utils.notebooks.experiment_storage)
import pickle
import os
from datetime import datetime

# Create output directory for storing experiment results (kept for compatibility)
OUTPUT_DIR = "experiment_results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

from utils.notebooks.experiment_storage import (
    list_available_samples as storage_list_available_samples,
    load_visualizations_data as storage_load_visualizations_data,
)

class LazyVisualizationLoader:
    """Lazy loader for visualization data - loads only what's needed when needed.

    This loader looks for files under:
        experiment_results/<sample_id>/after_viz_generation.pkl
    and falls back to older flat filenames when present.
    """

    def __init__(self, sample_id, output_dir=OUTPUT_DIR):
        self.sample_id = sample_id
        self.output_dir = output_dir
        self._metadata = None
        self._full_data = None
        self._loaded_combinations = {}

    def _ensure_loaded(self):
        if self._full_data is None:
            # Use the central storage loader which already handles fallbacks
            self._full_data = storage_load_visualizations_data(self.sample_id, self.output_dir)

    def load_metadata(self):
        """Load only the metadata (fast - no visualizations)"""
        if self._metadata is not None:
            return self._metadata

        self._ensure_loaded()
        data = self._full_data

        self._metadata = {
            'sample_id': data['sample_id'],
            'original_sample': data.get('original_sample'),
            'constraints': data.get('constraints', {}),
            'features_names': data.get('features_names', []),
            'target_class': data.get('target_class'),
            'num_combinations': len(data.get('visualizations', [])),
            'combination_labels': [viz.get('label') for viz in data.get('visualizations', [])],
            'combination_replication_counts': [len(viz.get('replication', [])) for viz in data.get('visualizations', [])]
        }

        return self._metadata

    def get_combination_data(self, combination_idx):
        """Load specific combination data on-demand"""
        if combination_idx in self._loaded_combinations:
            return self._loaded_combinations[combination_idx]

        self._ensure_loaded()

        try:
            combo_data = self._full_data['visualizations'][combination_idx]
        except Exception as e:
            raise IndexError(f"Combination {combination_idx} not found for sample {self.sample_id}") from e

        self._loaded_combinations[combination_idx] = combo_data
        return combo_data


def load_visualizations_data_lazy(sample_id):
    """Create a lazy loader for the sample"""
    loader = LazyVisualizationLoader(sample_id)
    loader.load_metadata()
    return loader


def list_available_samples():
    """List all available samples using the central storage helper."""
    return storage_list_available_samples(OUTPUT_DIR)

print(f"Output directory: {OUTPUT_DIR}")
print(f"Available samples: {len(list_available_samples())}")

## Setup + Constants

In [ ]:
CLASS_COLORS_LIST = ['purple', 'green', 'orange']
IRIS: Bunch = cast(Bunch, load_iris())
IRIS_FEATURES = IRIS.data
IRIS_LABELS = IRIS.target

TRAIN_FEATURES, TEST_FEATURES, TRAIN_LABELS, TEST_LABELS = train_test_split(IRIS_FEATURES, IRIS_LABELS, test_size=0.3, random_state=42)

MODEL = RandomForestClassifier(n_estimators=3, random_state=42)
MODEL.fit(TRAIN_FEATURES, TRAIN_LABELS)





## Load Data from Disk

In [ ]:
from ipywidgets import Dropdown, VBox, Output, HTML, Layout
from IPython.display import display, clear_output

# Get available samples
available_samples = list_available_samples()

if not available_samples:
    print("No samples found. Please run the previous cells to generate data first.")
else:
    # Create dropdown options from available samples
    dropdown_options = []
    for sample_id, metadata in sorted(available_samples.items()):
        original_sample = metadata['original_sample']
        predicted_class = metadata['predicted_class']
        target_class = metadata['target_class']
        sample_index = metadata.get('sample_index', 'N/A')
        timestamp = metadata.get('timestamp', 'N/A')
        
        # Format sample features
        feature_str = " | ".join([f"{k}: {v:.2f}" for k, v in original_sample.items()])
        label = f"Sample {sample_id} | idx:{sample_index} | {feature_str} | pred:{predicted_class} | target:{target_class}"
        dropdown_options.append((label, sample_id))
    
    print(f"Found {len(dropdown_options)} available samples")
    print("Select a sample below to explore its counterfactual combinations and replications")

In [ ]:
# Interactive visualization explorer with hierarchical selection: Sample -> Combination -> Replication
from ipywidgets import Dropdown, Output, VBox, HTML, Layout, IntSlider, HBox

# Check if samples are available
available_samples = list_available_samples()

print("available_samples:", available_samples.items())

if not available_samples:
    print("⚠ No samples found. Please run the experiment generation cells first.")
else:
    # Build sample dropdown options
    dropdown_options = []
    for sample_id, metadata in sorted(available_samples.items()):
        original_sample = metadata['original_sample']
        predicted_class = metadata['predicted_class']
        target_class = metadata['target_class']
        sample_index = metadata.get('sample_index', 'N/A')
        
        feature_str = " | ".join([f"{k}: {v:.2f}" for k, v in original_sample.items()])
        label = f"Sample {sample_id} | idx:{sample_index} | {feature_str} | pred:{predicted_class} | target:{target_class}"
        dropdown_options.append((label, sample_id))
    
    # Create sample selector dropdown
    sample_dropdown = Dropdown(
        options=dropdown_options,
        value=dropdown_options[0][1] if dropdown_options else None,
        description='Sample:',
        layout=Layout(width='100%'),
        style={'description_width': 'initial'}
    )
    
    # Define color mapping for rules
    RULE_COLORS = {
        'no_change': '#FF6B6B',  # Red
        'non_increasing': '#4ECDC4',  # Teal
        'non_decreasing': '#123456'  # blue
    }

    # Create combination slider
    combination_slider = IntSlider(
        value=0,
        min=0,
        max=0,
        step=1,
        description='Combination:',
        layout=Layout(width='500px')
    )

    # Create sample info display
    sample_info = HTML(value="", layout=Layout(margin='10px 0'))

    # Create combined label to show features and current rules
    combined_label = HTML(value="")

    # Create replication slider
    replication_slider = IntSlider(
        value=0,
        min=0,
        max=0,
        step=1,
        description='Replication:',
        layout=Layout(width='500px')
    )

    # Create output areas
    loading_status = HTML(value="")  # Loading indicator
    combination_output_area = Output()  # For PCA and Pairwise plots
    replication_output_area = Output()  # For replication visualizations

    # Slider labels for max values
    combination_slider_label = HTML(value="/ 0", layout=Layout(width='auto', margin='5px 0 0 10px'))
    replication_slider_label = HTML(value="/ 0", layout=Layout(width='auto', margin='5px 0 0 10px'))

    # Global state for current sample - store as dict to avoid closure issues
    STATE = {
        'loader': None,
        'metadata': None,
        'available_samples': available_samples,  # Store in state for access in callbacks
        'loader_cache': {},  # Cache loaders by sample_id to avoid reloading
        'is_loading': False,  # Guard flag to prevent re-entrant calls
    }

    def create_combined_label(combination_idx, features_names, combination_labels):
        """Create a label combining features and their corresponding rules with color coding"""
        if combination_idx >= len(combination_labels):
            return "<b>No visualizations available</b>"
        
        rules_tuple = combination_labels[combination_idx]
        label_parts = []
        for feat, rule in zip(features_names, rules_tuple):
            color = RULE_COLORS.get(rule, '#000000')
            label_parts.append(f"<b>{feat}=</b><span style='color: {color}; font-weight: bold;'>{rule}</span>")
        
        return "<br>".join(label_parts)

    def display_combination_plots(combination_idx):
        """Display PCA and Pairwise plots for selected combination - loads on demand"""
        combination_output_area.clear_output(wait=True)
        with combination_output_area:
            loading_status.value = "⏳ Loading combination data..."
            
            # Lazy load the combination data
            combination_viz = STATE['loader'].get_combination_data(combination_idx)
            
            loading_status.value = ""

            if combination_viz.get('pca') is not None:
                display_figure(combination_viz['pca'])

            if combination_viz.get('pairwise') is not None:
                display_figure(combination_viz['pairwise'])

    def display_replication(combination_idx, replication_idx):
        """Display visualizations for selected replication - loads on demand"""
        replication_output_area.clear_output(wait=True)
        with replication_output_area:
            loading_status.value = "⏳ Loading replication data..."
            
            # Lazy load the combination data (cached if already loaded)
            combination_viz = STATE['loader'].get_combination_data(combination_idx)
            
            loading_status.value = ""
            
            if replication_idx >= len(combination_viz['replication']):
                print(f"⚠ Replication {replication_idx} not found")
                return
                
            replication_viz = combination_viz['replication'][replication_idx]
            
            print(f"Replication {replication_idx + 1}:")

            # Display explanations
            explanations = replication_viz.get('explanations', {})
            for explanation_name, explanation_value in explanations.items():
                print(f"\n{explanation_name}:")
                # Special formatting for Feature Modifications (list of dicts)
                if explanation_name == 'Feature Modifications' and isinstance(explanation_value, list):
                    for mod in explanation_value:
                        feature_name = mod['feature_name']
                        old_value = mod['old_value']
                        new_value = mod['new_value']
                        # Get constraints for this feature from target class
                        target_class = STATE['metadata']['target_class']
                        constraints = STATE['metadata']['constraints']
                        target_class_constraints = constraints.get(f'Class {target_class}', [])
                        # Convert feature name to match constraint format
                        feature_key = feature_name.replace(' (cm)', '').replace(' ', '_')
                        # Find the constraint for this feature
                        feature_constraint = next((c for c in target_class_constraints if c['feature'] == feature_key), {})
                        min_val = feature_constraint.get('min', None)
                        max_val = feature_constraint.get('max', None)
                        constraint_str = f"(min: {min_val} -> max: {max_val})" if min_val is not None or max_val is not None else ""
                        print(f"  Feature '{feature_name}': {old_value} → {new_value} {constraint_str}")
                else:
                    print(explanation_value)

            # Use nested visualizations list
            for viz_idx, viz in enumerate(replication_viz.get('visualizations', [])):
                display_figure(viz)

    def load_sample(sample_id):
        """Load a sample and update all UI components"""
        
        # Guard against re-entrant calls
        if STATE['is_loading']:
            return
        STATE['is_loading'] = True
        
        try:
            # Check if we already have this sample cached
            if sample_id in STATE['loader_cache']:
                loading_status.value = f"✓ Loading cached sample {sample_id}..."
                STATE['loader'] = STATE['loader_cache'][sample_id]
                STATE['metadata'] = STATE['loader']._metadata
            else:
                loading_status.value = f"⏳ Loading sample {sample_id}..."
                try:
                    # Create lazy loader and load metadata
                    STATE['loader'] = load_visualizations_data_lazy(sample_id)
                    STATE['metadata'] = STATE['loader']._metadata
                    # Cache the loader for future use
                    STATE['loader_cache'][sample_id] = STATE['loader']
                except Exception as e:
                    loading_status.value = f"✗ Error loading sample {sample_id}: {str(e)}"
                    print(f"Error: {e}")
                    import traceback
                    traceback.print_exc()
                    return
            
            # Get sample metadata from available_samples for additional info
            sample_metadata = STATE['available_samples'].get(sample_id, {})
            
            # Build sample info display
            original_sample = STATE['metadata']['original_sample']
            target_class = STATE['metadata']['target_class']
            predicted_class = sample_metadata.get('predicted_class', 'N/A')
            sample_index = sample_metadata.get('sample_index', 'N/A')
            timestamp = sample_metadata.get('timestamp', 'N/A')
            
            # Format sample features
            feature_lines = []
            for feat_name, feat_value in original_sample.items():
                feature_lines.append(f"<b>{feat_name}:</b> {feat_value:.4f}")
            
            sample_info.value = f"""
            <div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px; margin: 10px 0;">
                <b>Sample Information:</b><br>
                <b>Sample ID:</b> {sample_id} | <b>Index:</b> {sample_index}<br>
                <b>Generated:</b> {timestamp}<br>
                <b>Predicted Class:</b> {predicted_class} → <b>Target Class:</b> {target_class}<br>
                <br>
                <b>Original Features:</b><br>
                {' | '.join(feature_lines)}
            </div>
            """
            
            # Update combination slider
            num_combinations = STATE['metadata']['num_combinations']
            combination_slider.max = max(0, num_combinations - 1)
            combination_slider.value = 0
            combination_slider_label.value = f"/ {combination_slider.max}"
            
            # Update combined label
            if num_combinations > 0:
                combined_label.value = create_combined_label(
                    0, 
                    STATE['metadata']['features_names'],
                    STATE['metadata']['combination_labels']
                )
                
                # Load first combination to get replication count
                combination_viz = STATE['loader'].get_combination_data(0)
                num_replications = len(combination_viz['replication'])
                
                # Update replication slider
                replication_slider.max = max(0, num_replications - 1)
                replication_slider.value = 0
                replication_slider_label.value = f"/ {replication_slider.max}"
                
                loading_status.value = f"✓ Sample {sample_id} loaded ({num_combinations} combinations, {sum(STATE['metadata']['combination_replication_counts'])} total replications)"
                
                # Display first combination and replication
                display_combination_plots(0)
                display_replication(0, 0)
            else:
                combined_label.value = "<b>No combinations available</b>"
                loading_status.value = f"✓ Sample {sample_id} loaded (no combinations)"
                
            # --- FIX: Set global variables for summary table cell ---
            global CURRENT_LAZY_LOADER, CURRENT_METADATA
            CURRENT_LAZY_LOADER = STATE['loader']
            CURRENT_METADATA = STATE['metadata']
            # ------------------------------------------------------
        except Exception as e:
            loading_status.value = f"✗ Error displaying sample {sample_id}: {str(e)}"
            print(f"Error: {e}")
            import traceback
            traceback.print_exc()
        finally:
            STATE['is_loading'] = False

    def on_sample_change(change):
        """Handle sample selection change"""
        sample_id = change['new']
        combination_output_area.clear_output()
        replication_output_area.clear_output()
        load_sample(sample_id)

    def on_combination_change(change):
        """Handle combination selection change"""
        # Skip if we're in the middle of loading a sample
        if STATE['is_loading']:
            return
            
        combination_idx = change['new']
        
        # Update combined label with features and rules
        combined_label.value = create_combined_label(
            combination_idx,
            STATE['metadata']['features_names'],
            STATE['metadata']['combination_labels']
        )
        
        # Get combination data to determine replication count
        combination_viz = STATE['loader'].get_combination_data(combination_idx)
        
        # Update replication slider range
        num_replications = len(combination_viz['replication'])
        replication_slider.max = max(0, num_replications - 1)
        replication_slider.value = 0
        replication_slider_label.value = f"/ {replication_slider.max}"
        
        # Display combination plots
        display_combination_plots(combination_idx)
        
        # Display first replication
        if num_replications > 0:
            display_replication(combination_idx, 0)

    def on_replication_change(change):
        """Handle replication selection change"""
        # Skip if we're in the middle of loading a sample
        if STATE['is_loading']:
            return
            
        combination_idx = combination_slider.value
        replication_idx = change['new']
        display_replication(combination_idx, replication_idx)

    # Register observers (fresh widgets don't need unobserve)
    sample_dropdown.observe(on_sample_change, names='value')
    combination_slider.observe(on_combination_change, names='value')
    replication_slider.observe(on_replication_change, names='value')

    # Wrap sliders with labels
    combination_slider_with_label = HBox([combination_slider, combination_slider_label])
    replication_slider_with_label = HBox([replication_slider, replication_slider_label])

    # Display all controls in a single VBox
    display(VBox([
        sample_dropdown,
        loading_status,
        sample_info,
        combined_label,
        combination_slider_with_label,
        combination_output_area,
        replication_slider_with_label,
        replication_output_area
    ]))

    # Auto-load first sample
    if dropdown_options:
        load_sample(dropdown_options[0][1])

## Summary Table of All Counterfactuals

In [ ]:
import pandas as pd
import json
from IPython.display import display, HTML
from ipywidgets import IntSlider, VBox, HBox, Output, Button, Layout, HTML as HTMLWidget

# Use current loaded data if available
if 'CURRENT_LAZY_LOADER' in globals() and CURRENT_LAZY_LOADER is not None:
    print(f"Building summary table for currently loaded sample...")
    print("⏳ This may take a moment as all data needs to be loaded...")
    
    FEATURES_NAMES = CURRENT_METADATA['features_names']
    
    # Build a comprehensive table from the visualizations data structure
    table_data = []

    for combination_idx in range(CURRENT_METADATA['num_combinations']):
        # Lazy load this combination
        combination_viz = CURRENT_LAZY_LOADER.get_combination_data(combination_idx)
        
        # Get the rules for this combination
        rules_tuple = combination_viz['label']
        rules_dict = dict(zip(FEATURES_NAMES, rules_tuple))
        
        # Iterate through each replication
        for replication_idx, replication_viz in enumerate(combination_viz['replication']):
            row = {
                'Combination': combination_idx + 1,
                'Replication': replication_idx + 1,
            }
            
            # Add rule columns
            for feature_name in FEATURES_NAMES:
                row[f'Rule_{feature_name}'] = rules_dict[feature_name]
            
            # Add counterfactual feature values
            counterfactual = replication_viz['counterfactual']
            for feature_name in FEATURES_NAMES:
                row[f'CF_{feature_name}'] = counterfactual.get(feature_name, None)
            
            # Add counts of visualizations and explanations
            row['Num_Visualizations'] = len(replication_viz.get('visualizations', []))
            row['Num_Explanations'] = len(replication_viz.get('explanations', {}))
            
            # Add explanation content from dictionary
            explanations = replication_viz.get('explanations', {})
            for explanation_name, explanation_value in explanations.items():
                # Special handling for Feature Modifications (list of dicts)
                if explanation_name == 'Feature Modifications' and isinstance(explanation_value, list):
                    # Format as readable HTML text: "feature, old => new (delta)"
                    formatted_mods = []
                    for mod in explanation_value:
                        feature_name = str(mod['feature_name'])
                        old_val = float(mod['old_value'])
                        new_val = float(mod['new_value'])
                        delta = new_val - old_val
                        formatted_mods.append(f"{feature_name}, {old_val} => {new_val} ({delta:+.2f})")
                    # Join with HTML <br> so it renders one per line in DataFrame HTML representation
                    row[explanation_name] = "<br>".join(formatted_mods)
                else:
                    row[explanation_name] = explanation_value
            
            # Extract any other keys that might exist in replication_viz
            for key in replication_viz.keys():
                if key not in ['counterfactual', 'cf_model', 'visualizations', 'explanations']:
                    row[key] = replication_viz[key]
            
            table_data.append(row)

    # Create DataFrame
    summary_df = pd.DataFrame(table_data)

    # Display summary statistics
    print(f"Total Combinations: {CURRENT_METADATA['num_combinations']}")
    print(f"Total Replications: {len(summary_df)}")
    if CURRENT_METADATA['num_combinations'] > 0:
        print(f"Average Replications per Combination: {len(summary_df) / CURRENT_METADATA['num_combinations']:.2f}")
    print("\n")

    # Display the summary table (rendering HTML so <br> shows as new lines)
    print("\n" + "=" * 150)
    print("DETAILED COUNTERFACTUAL SUMMARY TABLE")
    print("=" * 150)
    display(HTML(summary_df.to_html(escape=False)))
else:
    print("⚠ No sample loaded. Please run the interactive visualization cell above and select a sample first.")

In [ ]:
import sys
sys.path.append('Scamander')

from cf_eval.metrics import (
    nbr_valid_cf,
    perc_valid_cf,
    continuous_distance,
    avg_nbr_changes_per_cf,
    nbr_changes_per_cf
)

# Check if we have loaded data
if 'CURRENT_LAZY_LOADER' in globals() and CURRENT_LAZY_LOADER is not None:
    print("="*80)
    print("COUNTERFACTUAL QUALITY METRICS EVALUATION")
    print("="*80)
    print()
    
    # Get original sample and metadata
    original_sample = CURRENT_METADATA['original_sample']
    target_class = CURRENT_METADATA['target_class']
    
    # Convert original sample dict to numpy array
    x_original = np.array([original_sample[feat] for feat in CURRENT_METADATA['features_names']])
    y_original = MODEL.predict(x_original.reshape(1, -1))[0]
    
    print(f"Original Sample: {original_sample}")
    print(f"Predicted Class: {y_original} → Target Class: {target_class}")
    print()
    
    # Collect all counterfactuals from all combinations
    all_metrics = []
    
    for combination_idx in range(CURRENT_METADATA['num_combinations']):
        combination_viz = CURRENT_LAZY_LOADER.get_combination_data(combination_idx)
        rules_tuple = combination_viz['label']
        
        # Collect counterfactuals for this combination
        cf_list = []
        for replication_idx, replication_viz in enumerate(combination_viz['replication']):
            cf_dict = replication_viz['counterfactual']
            cf_array = np.array([cf_dict[feat] for feat in CURRENT_METADATA['features_names']])
            cf_list.append(cf_array)
        
        if len(cf_list) == 0:
            continue
        
        cf_array = np.array(cf_list)
        
        # ====================
        # METRIC 1: Validity
        # ====================
        # Number and percentage of counterfactuals that achieve the desired class
        num_valid = nbr_valid_cf(cf_array, MODEL, y_original, y_desidered=target_class)
        pct_valid = perc_valid_cf(cf_array, MODEL, y_original, y_desidered=target_class)
        
        # ====================
        # METRIC 2: Proximity (Distance)
        # ====================
        # How close are the counterfactuals to the original sample (lower is better)
        # Using Euclidean distance on continuous features (all Iris features are continuous)
        continuous_features = list(range(len(CURRENT_METADATA['features_names'])))
        avg_distance = continuous_distance(x_original, cf_array, continuous_features, metric='euclidean')
        min_distance = continuous_distance(x_original, cf_array, continuous_features, metric='euclidean', agg='min')
        max_distance = continuous_distance(x_original, cf_array, continuous_features, metric='euclidean', agg='max')
        
        # ====================
        # METRIC 3: Sparsity (Number of Changes)
        # ====================
        # How many features were changed (fewer is better - more actionable)
        avg_changes = avg_nbr_changes_per_cf(x_original, cf_array, continuous_features)
        changes_per_cf = nbr_changes_per_cf(x_original, cf_array, continuous_features)
        
        all_metrics.append({
            'Combination': combination_idx + 1,
            'Rules': str(rules_tuple),
            'Num_CFs': len(cf_list),
            'Valid_CFs': num_valid,
            'Validity_%': f"{pct_valid*100:.1f}%",
            'Avg_Distance': f"{avg_distance:.4f}",
            'Min_Distance': f"{min_distance:.4f}",
            'Max_Distance': f"{max_distance:.4f}",
            'Avg_Changes': f"{avg_changes:.2f}",
            'Changes_Detail': [f"{c:.1f}" for c in changes_per_cf]
        })
    
    # Create DataFrame and display
    metrics_df = pd.DataFrame(all_metrics)
    
    print("\n" + "="*80)
    print("METRICS BY COMBINATION")
    print("="*80)
    display(HTML(metrics_df.to_html(escape=False, index=False)))
    
    print("\n" + "="*80)
    print("METRIC EXPLANATIONS")
    print("="*80)
    print("""
    1. Validity: Percentage of counterfactuals that successfully achieve the target class
       - Higher is better (100% means all CFs are valid)
    
    2. Proximity (Distance): Average Euclidean distance from original sample
       - Lower is better (smaller changes from original)
    
    3. Sparsity (Avg Changes): Average number of features modified per counterfactual
       - Lower is better (fewer features changed = more actionable)
    """)
    
else:
    print("⚠ No sample loaded. Please run the interactive visualization cells above first.")